<img src="https://raw.githubusercontent.com/OpenEnergyPlatform/academy/develop/docs/data/img/OEP_logo_2_no_text.svg" alt="OpenEnergy Platform" height="75" width="75" align="left"/>

# Usage of the oedialect a python sqlalchemy dialect for the OEP 

Repository: https://github.com/openego/oedialect <br>
Documentation: http://oep-data-interface.readthedocs.io/en/latest/api/how_to.html<br>
Please report bugs and improvements here: https://github.com/OpenEnergyPlatform/oedialect/issues <br>

In [1]:
__copyright__ = "Reiner Lemoine Institut"
__license__   = "GNU Affero General Public License Version 3 (AGPL-3.0)"
__url__       = "https://github.com/openego/data_processing/blob/master/LICENSE"
__author__    = "henhuy, Ludee, oakca"

In [2]:
# pip install oedialect "sqlalchemy~=1.3,<1.4" pandas

import os
import getpass
import pandas as pd
import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker
import oedialect

## Connection to OEP

If we want to upload data to the OEP we first need to connect to it, using our OEP user name and token. 
Note: You ca view your token on your OEP profile page after logging in.

In [3]:
# set your token, for example from an environment variable
# NEVER commit your token to a repository

# token = getpass.getpass('Token:')  # get token from user input
token = os.environ["OEP_API_TOKEN"]  # OR get token from environment variable

In [4]:
# Create Engine:
OEP_URL = 'openenergy-platform.org'
OED_STRING = f'postgresql+oedialect://:{token}@{OEP_URL}'

engine = sa.create_engine(OED_STRING)
metadata = sa.MetaData(engine)

## Setup a Table

We need to tell the data base what columns and datatypes we are about to upload. In our case we have four columns, two of which are text, one is integer and the last is float.

In [5]:
table_name = 'example_dialect_tablex'
schema_name = 'sandbox'

ExampleTable = sa.Table(
    table_name,
    metadata,
    sa.Column('id', sa.BIGINT()),
    sa.Column('name', sa.VARCHAR(50)),
    sa.Column('age', sa.INTEGER),
    sa.Column('stadtname', sa.VARCHAR(50)),
    schema=schema_name
)

## Create the new Table
Now we tell our engine to connect to the data base and create the defined table within the chosen schema.

In [6]:
conn = engine.connect()
print('Connection established')
if not engine.dialect.has_table(conn, table_name, schema_name):
    ExampleTable.create()
    print('Created table')
else:
    print('Table already exists')

Connection established
Created table


## Insert data into Table
Uploading the information from our DataFrame is now done with a single command. Uploading data in this way will always delete the content of the table and refill it with new values every time. If you change 'replace' to 'append', the data entries will be added to the preexisting ones. (Connecting and uploading may take a minute.)

In [7]:
Session = sessionmaker(bind=engine)
session = Session()
try:
    insert_statement = ExampleTable.insert().values(
        [
            dict(name='Carsten', age=25, stadtname='Berlin'),
            dict(name='Bert', age=42, stadtname='Hamburg'),
            dict(name='Rhamses', age=69, stadtname='München')
        ]
    )
    session.execute(insert_statement)
    session.commit()
    print('Insert successful!')
except Exception as e:
    session.rollback()
    print('Insert incomplete!')
    raise    
finally:
    session.close()

Insert successful!


## Select from Table

Now  we can query our table to see if the data arrived.

In [8]:
Session = sessionmaker(bind=engine)
session = Session()
print(session.query(ExampleTable).all())
session.close()

[(1, 'Carsten', 25, 'Berlin'), (2, 'Bert', 42, 'Hamburg'), (3, 'Rhamses', 69, 'München')]


In [9]:
Session = sessionmaker(bind=engine)
session = Session()
df = pd.DataFrame(session.query(ExampleTable).all())
df = df.set_index("id")
session.close()
df

,name,age,stadtname
id,,,
1,Carsten,25,Berlin
2,Bert,42,Hamburg
3,Rhamses,69,München


# Delete Table

In [10]:
ExampleTable.drop()